In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [2]:
data = pd.read_excel("/content/sample_data/Load_Data_in_kW.xlsx",header=None)
data.head()

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980


In [3]:
z = np.abs(stats.zscore(data))
print(z)

[[0.04910416]
 [0.08229029]
 [0.08782131]
 ...
 [0.09743692]
 [0.07253822]
 [0.03181432]]


In [4]:
df = data
q = df[0].quantile(0.99)
df[df[0] < q]
q_low = df[0].quantile(0.01)
q_hi  = df[0].quantile(0.99)

df_filtered = df[(df[0] < q_hi) & (df[0] > q_low)]
df_filtered.head()

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980


In [5]:
print(len(data))
print(len(df_filtered))
if(len(data) == len(df_filtered)):
    print('No outliners')

2184
2140


In [6]:
data = df_filtered
data_n = data.copy()
data_n = (data_n - data_n.min())/(data_n.max() - data_n.min())
data_n.head()

,0
0,0.372950
1,0.254941
2,0.235273
3,0.273574
4,0.299162


In [9]:
train_data,test_data = train_test_split(data_n,test_size=0.1)
print(len(train_data))
train_data.head()

1926


,0
2148,0.105425
911,0.328406
2178,0.267881
403,0.622812
1714,0.506292


In [10]:
xar = []
yar = []
for i in range(len(train_data.iloc[:,0])-24):
    xar.append(train_data.iloc[i,0])
    yar.append(train_data.iloc[i+24,0])

In [11]:
from sympy import *
m, c = symbols('m c')
exp = '0.5*(y-m*x-c)**2'
exp_diff = Derivative(exp,m)
dfm = exp_diff.doit()
exp_diff = Derivative(exp,c)
dfc = exp_diff.doit()
print(f'1st order derivative w.r.t m and c :\nde/dm = {dfm} \nde/dc = {dfc}')
err=[]

#step 1:
m1=1.0
c1=-1.0
learning_rate=0.1
itr=25
g=0.9
vm = vc = 0

for i in range(0,itr):
  er=0
  for j in range(0,len(xar)):
    er +=  ((yar[j] - (m1 * xar[j]) - c1))**2
    #step 2:
    m,c,x,y = symbols('m c x y')
    dedm = dfm.subs(m,m1+g*vm)
    dedm = dedm.subs(c, c1+g*vc) 
    dedm= dedm.subs(x, xar[j]) 
    dedm = dedm.subs(y, yar[j])
    dedm = round(dedm,2)
    dedc = dfc.subs(c,c1+g*vc)
    dedc = dedc.subs(m, m1+g*vm) 
    dedc = dedc.subs(x, xar[j]) 
    dedc = dedc.subs(y, yar[j])
    dedc = round(dedc,2)
    #step 3:
    vm = (g*vm)-learning_rate*dedm
    vc = (g*vc)-learning_rate*dedc
    #step 4:
    m1 = m1+vm
    m1 = round(m1,2)
    c1 = c1+vc
    c1 = round(c1,2)
    er = (1/(2*len(xar)))*(er)
    err.append(er)

print(f'min value of m = {m1} \nmin value of c = {c1}')

1st order derivative w.r.t m and c :
de/dm = -1.0*x*(-c - m*x + y) 
de/dc = 1.0*c + 1.0*m*x - 1.0*y
min value of m = -0.14 
min value of c = 0.46


In [13]:
train_data_pred = [i for i in xar]
for k in range(len(xar)):
    train_data_pred[k]=(m1*xar[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,train_data_pred)
mse

0.05084127168214396

In [14]:
xar = []
yar = []
for i in range(len(test_data.iloc[:,0])-24):
    xar.append(test_data.iloc[i,0])
    yar.append(test_data.iloc[i+24,0])
test_data_pred = [i for i in xar]
for k in range(len(xar)):
    test_data_pred[k]=(m1*xar[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,test_data_pred)
mse

0.05280731837652227

In [16]:
x_max = y_max= 8841.669480
x_min = y_min = 3377.919600
x =x1 = float(input("enter load of previous hour :"))
x = (x - x_min)/(x_max - x_min)
y = m1*x + c1
y = y*(y_max - y_min) + y_min
print(f"load at hour based on its previous hour is : {y}")

enter load of previous hour :5072.95
load at hour based on its previous hour is : 5653.90894441406
